# Compress DICOMs using JPEG 2000 Lossless compression
- take a large 1.9GB DICOM multi-frame 3d scan DICOM and compress it
- HTJ2K lossless compression
- NVImage library

### Requirements
- Transcoding: `nvidia-nvimgcodec-cu12[all] pydicom==3.0.1 cupy-cuda12x>=13.5.0 pylibjpeg[all]`
- Indexing: `pylibjpeg>=2.0 pylibjpeg-openjpeg>=2.0`

Tested on Databricks Accelerated Serverless Compute, A10 GPU; Env 3

In [0]:
%pip install --quiet numpy==1.26.4 nvidia-nvimgcodec-cu12[all] pydicom==3.0.1 cupy-cuda12x>=13.5.0 pylibjpeg[all]
%pip install mlflow[skinny]

In [0]:
dbutils.library.restartPython()

In [0]:
#
# Pull configuration from config.yaml
#

import yaml
import pprint
compression = "nvImage_HTJ2K"
cfg = yaml.safe_load(open("config.yaml", "r"))
input_path = cfg.get("input_path")
output_path = cfg.get("output_path").replace("{compression}", f"{compression}")
table = cfg.get("table")

tempfile = "/tmp/compressed.dcm"

pprint.pprint(cfg, indent=4)
print(output_path)

In [0]:
import pydicom
import numpy as np
import cupy as cp
from nvidia import nvimgcodec
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000Lossless  # or JPEG2000 for lossy
import time
import torch
print(f"Has GPU: {torch.cuda.is_available()}")
print(f"#GPUs: {torch.cuda.device_count()}")
import os
#os.environ["PYLIBJPEG_OPENJPEG"] = "1"



## Encode DICOM

In [0]:
#
# encoder setup
# (default works amazingly well)

encoded_frames = []
encoder = nvimgcodec.Encoder()

htj2k_params = nvimgcodec.Jpeg2kEncodeParams(
    ht=True,
#    num_resolutions=1
)

enc_params = nvimgcodec.EncodeParams(jpeg2k_encode_params=htj2k_params)


print(f"{enc_params.jpeg2k_params.ht=}")
print(f"{enc_params.jpeg2k_params.num_resolutions=}")


In [0]:
if True or torch.cuda.is_available():
    start = time.time()
    # Step 1: Load the DICOM
    # read DICOM image
    ds = pydicom.dcmread(input_path)
    print(f"Starting value {ds.file_meta.TransferSyntaxUID=}")
    ds.pixel_array_options(use_v2_backend=True)
    pixel_array = ds.pixel_array  # shape: (frames, rows, cols)


    # Step 2: For each frame, move to GPU, encode, and store results
    for i in range(pixel_array.shape[0]):
        frame = pixel_array[i]  # NumPy CPU array, shape (rows, cols)
        frame_cp = cp.asarray(frame[:, :, np.newaxis])  # shape (rows, cols, 1), CuPy GPU array
        nv_img = nvimgcodec.as_image(frame_cp.astype(cp.uint16))  # pass CuPy array directly
        j2k_bytes = encoder.encode(nv_img, "jpeg2k", params=enc_params)
        encoded_frames.append(j2k_bytes)

    # Step 3: Encapsulate for DICOM
    ds.PixelData = encapsulate(encoded_frames)
    ds['PixelData'].is_undefined_length = True


    # Step 4: Set correct Transfer Syntax UID for HTJ2K (lossless)
    ds.file_meta.TransferSyntaxUID = "1.2.840.10008.1.2.4.201"

    shutil.os.remove(output_path)
    ds.save_as(output_path)

    duration = time.time() - start
    print(f"Saved to {output_path} \nduration: {duration:.2f} seconds")

In [0]:
# report stats
output_size = os.stat(output_path).st_size
input_size = os.stat(input_path).st_size

print(f"{input_path}")
print(f"{output_path}")
print(f"Input  size: {input_size:>15,}")
print(f"Output size: {output_size:>15,}")
print(f"Reduction:   {round(input_size/output_size,2):>18.2f}x")
print(f"Savings:     {round(100*(input_size-output_size)/input_size,2):>18.2f}%")
print(f"Time taken:  {round(duration,2):>18.2f} seconds")

## Decode


In [0]:
# test decode
import pydicom
import pylibjpeg
ds = pydicom.dcmread(output_path)
pixel_array = ds.pixel_array
pixel_array.shape

In [0]:
from pydicom.dataset import validate_file_meta

# Validate file meta information
try:
    validate_file_meta(ds.file_meta, enforce_standard=True)
    print("File meta information is valid")
except Exception as e:
    print(f"File meta validation error: {e}")